In [1]:
import tarfile
import urllib.request
import tempfile
import anndata as ad
import scanpy as sc

import pandas as pd
import numpy as np
import seaborn as sb
from scipy import io, sparse
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import pickle
import copy

import rpy2.rinterface_lib.callbacks
import logging
from rpy2.robjects import pandas2ri
import anndata2ri
%load_ext rpy2.ipython

In [2]:
%%R
suppressMessages(library("dplyr"))
suppressMessages(library("Seurat"))
suppressMessages(library("anndata"))
suppressMessages(library("SingleCellExperiment"))
suppressMessages(library("scran"))
suppressMessages(library("Matrix"))

In [3]:
## VIASH START
par = {
    "id": "totalVI_x_malt_k",
    "input": "https://github.com/YosefLab/totalVI_reproducibility/raw/master/data/malt_10k_protein_v3.h5ad",
    "organism": "human",
    "output_rna": "output_rna.h5ad",
    "output_mod2": "output_mod2.h5ad"
}
## VIASH END

###############################################################################
###                     DOWNLOAD AND READ DATA.                             ###
###############################################################################
print("Downloading file from", par['input'])
h5ad_temp = tempfile.NamedTemporaryFile()
url = par['input']
urllib.request.urlretrieve(url, h5ad_temp.name)

print("Reading h5ad file")
adata = sc.read_h5ad(h5ad_temp.name)
h5ad_temp.close()

###############################################################################
###                     CREATE H5AD FOR BOTH MODALITIES                     ###
###############################################################################
uns = { "dataset_id" : par["id"], "organism" : par["organism"] }

print("Extracting RNA counts")
adata_RNA = ad.AnnData(
    X=adata.X,
    obs=adata.obs.loc[:,['n_genes', 'percent_mito', 'n_counts']],
    var=adata.var,
    uns=uns
)

adata_RNA.var['feature_types'] = "GEX"

print("Extracting ADT counts")
adata_ADT = ad.AnnData(
    X=adata.obsm['protein_expression'],
    var=pd.DataFrame(index=list(adata.uns['protein_names'])),
    uns=uns
)

adata_ADT.obs.index = adata.obs.index
adata_ADT.var['feature_types'] = "ADT"

###############################################################################
###                             SAVE OUTPUT                                 ###
###############################################################################

# Convert to sparse matrices
adata_RNA.X = sparse.csr_matrix(adata_RNA.X)
adata_ADT.X = sparse.csr_matrix(adata_ADT.X)

print("Saving output")
adata_RNA.write_h5ad(par['output_rna'], compression = "gzip")
adata_ADT.write_h5ad(par['output_mod2'], compression = "gzip")

Reading h5ad file
Extracting RNA counts
Extracting ADT counts


/home/alina/Downloads/ENTER/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'feature_types' as categorical


Saving output


/home/alina/Downloads/ENTER/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'feature_types' as categorical


# Quality Control metrics

In [4]:
# pct_counts_mt
mito_genes = adata_RNA.var_names.str.startswith('MT-')
adata_RNA.obs['pct_counts_mt'] = (np.sum(adata_RNA[:, mito_genes].X, axis=1).A1 / np.sum(adata_RNA.X, axis=1).A1)*100
adata_RNA.obs['pct_counts_mt'].max()

14.99221134185791

In [5]:
# UMI counts per cell
# rows: n of cells = 7361; columns: n of genes = 36601
# asarray convert matrix (7361, 1) to array (7361,)
adata_RNA.obs['n_counts'] = np.asarray(np.sum(adata_RNA.X, axis = 1)).reshape(-1)
adata_RNA.obs['n_counts'].min()

590.0

In [6]:
# n of genes per cell
adata_RNA.obs['n_genes'] = np.asarray((adata_RNA.X > 0).sum(axis=1)).reshape(-1)
adata_RNA.obs['n_genes'].min()

337

In [8]:
# Copy AnnData object

adata_RNA_filtered = copy.copy(adata_RNA)
adata_RNA_filtered.shape

(6838, 16659)

In [9]:
# Filter cells according to identified QC thresholds:

print('Total number of cells: {:d}'.format(adata_RNA_filtered.n_obs))

sc.pp.filter_cells(adata_RNA_filtered, min_counts = 1500)

print('Number of cells after min count filter: {:d}'.format(adata_RNA_filtered.n_obs))

sc.pp.filter_cells(adata_RNA_filtered, max_counts = 40000)
print('Number of cells after max count filter: {:d}'.format(adata_RNA_filtered.n_obs))

adata_RNA_filtered = adata_RNA_filtered[adata_RNA_filtered.obs['pct_counts_mt'] < 20]
print('Number of cells after MT filter: {:d}'.format(adata_RNA_filtered.n_obs))

sc.pp.filter_cells(adata_RNA_filtered, min_genes = 700)
print('Number of cells after gene filter: {:d}'.format(adata_RNA_filtered.n_obs))

Trying to set attribute `.obs` of view, copying.


Total number of cells: 6838
Number of cells after min count filter: 6781
Number of cells after max count filter: 6781
Number of cells after MT filter: 6781
Number of cells after gene filter: 6765


In [10]:
# Filter genes

print('Total number of genes: {:d}'.format(adata_RNA.n_vars))

Total number of genes: 16659


In [11]:
# Min 20 cells: filters out 0 count genes

sc.pp.filter_genes(adata_RNA_filtered, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(adata_RNA_filtered.n_vars))

Number of genes after cell filter: 12629


In [12]:
adata_RNA.write('./adata_RNA_mt.h5ad')
adata_RNA_filtered.write('./adata_RNA_filtered.h5ad')

# Normalization (size factors)

In [13]:
# Perform a clustering for scran normalization in clusters

adata_pp = adata_RNA_filtered.copy()

sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e6)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=15)
sc.pp.neighbors(adata_pp)
sc.tl.louvain(adata_pp, key_added='groups', resolution=0.5)

In [14]:
# Preprocess variables for scran normalization

input_groups = adata_pp.obs['groups']
data_mat = adata_RNA_filtered.X.T
io.mmwrite("data_mat.mtx", data_mat, comment='', field=None, precision=None, symmetry=None)

In [15]:
%%R -i input_groups -o size_factors

data_mat <- readMM("./data_mat.mtx")
size_factors = calculateSumFactors(data_mat, clusters=input_groups, min.mean=0.1)

In [16]:
adata_RNA_filtered.obs['size_factors'] = size_factors

In [17]:
# Keep the count data in a counts layer
adata_RNA_filtered.layers["counts"] = adata_RNA_filtered.X.copy()

In [18]:
# Normalize & Log-transform 
adata_RNA_filtered.X /= adata_RNA_filtered.obs['size_factors'].values[:,None]
adata_RNA_filtered.layers["log_norm"] = sc.pp.log1p(adata_RNA_filtered.X)

In [19]:
adata_RNA_filtered

AnnData object with n_obs × n_vars = 6765 × 12629
    obs: 'n_genes', 'percent_mito', 'n_counts', 'pct_counts_mt', 'size_factors'
    var: 'n_cells', 'highly_variable', 'feature_types'
    uns: 'dataset_id', 'organism'
    layers: 'counts', 'log_norm'

In [63]:
adata_RNA_filtered.write("adata_RNA_filtered")